# Binary CSP solver

*Guillaume DALLE*

## 1. CSP structure and backtracking

In [1]:
mutable struct CSP
    n_variables::Int64
    domain::Array{Array{Int64}, 1}
    possible_value::Dict{Tuple{Int64, Int64}, Bool}
        
    n_constraints::Int64
    constraint_var1::Array{Int64, 1}
    constraint_var2::Array{Int64, 1}
    constraint_satisfaction::Array{Array{Bool, 2}, 1}
    
    var1_constraints::Array{Array{Int64, 1}, 1}
    var2_constraints::Array{Array{Int64, 1}, 1}
    
    CSP() = new(
        0, [], Dict{Tuple{Int64, Int64}, Bool}(),
        0, [], [], [],
        [], [],
    )
end

In [2]:
function add_variable!(csp::CSP, domain::Array{Int64})
    csp.n_variables += 1
    push!(csp.domain, domain)
    push!(csp.var1_constraints, Array{Int64, 1}(undef, 0))
    push!(csp.var2_constraints, Array{Int64, 1}(undef, 0))
    for value in domain
        csp.possible_value[(csp.n_variables, value)] = true
    end
end

add_variable! (generic function with 1 method)

In [3]:
function add_constraint!(csp::CSP, var1::Int64, var2::Int64, compatibility::Function)
    csp.n_constraints += 1

    m1::Int64 = length(csp.domain[var1])
    m2::Int64 = length(csp.domain[var2])
    satisfaction::Array{Bool, 2} = zeros(Int64, m1, m2)
    for value1 in csp.domain[var1], value2 in csp.domain[var2]
        satisfaction[value1, value2] = compatibility(var1, var2, value1, value2)
    end
    
    push!(csp.constraint_var1, var1)
    push!(csp.constraint_var2, var2)
    push!(csp.constraint_satisfaction, satisfaction)
    
    push!(csp.var1_constraints[var1], csp.n_constraints)
    push!(csp.var2_constraints[var2], csp.n_constraints)
end

add_constraint! (generic function with 1 method)

In [4]:
function check_feasibility(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    if new_var == 0
        return true
    end
    for cons in csp.var1_constraints[new_var]
        var2::Int64 = csp.constraint_var2[cons]
        satisfaction = csp.constraint_satisfaction[cons]
        value1 = instantiation[new_var]
        value2 = instantiation[var2]
        if (value1 !== 0) && (value2 !== 0)
            if !satisfaction[value1, value2]
                return false
            end
        end
    end
    return true
end

check_feasibility (generic function with 1 method)

In [5]:
function undo_look_ahead!(csp::CSP, discarded::Dict{Int64, Array{Int64, 1}})
    for (var2, discarded_var2) in discarded
        for value2 in discarded_var2
            csp.possible_value[(var2, value2)] = true
        end
    end
end

undo_look_ahead! (generic function with 1 method)

In [6]:
function forward_checking(csp::CSP, instantiation::Array{Int64, 1}, x::Int64)
    to_discard::Dict{Int64, Array{Int64, 1}} = Dict{Int64, Array{Int64}}()
    
    a::Int64 = instantiation[x]
    to_discard[x] = Array{Int64, 1}()
    for aa in csp.domain[x]
        if (aa != a) && csp.possible_value[(x, aa)]
            push!(to_discard[x], aa)
        end
    end
    
    for cons in csp.var1_constraints[x]
        y = csp.constraint_var2[cons]
        if instantiation[y] != 0
            continue
        end
        
        to_discard[y] = Array{Int64, 1}()
        for b in csp.domain[y]
            if (!csp.constraint_satisfaction[cons][a, b]) && csp.possible_value[(y, b)]
                push!(to_discard[y], b)
            end
        end
    end
    
    for (var2, discarded_var2) in to_discard
        for value2 in discarded_var2
            csp.possible_value[(var2, value2)] = false
        end
    end
    
    return to_discard
end

forward_checking (generic function with 1 method)

In [7]:
function AC3(csp::CSP, instantiation::Array{Int64, 1}, new_var::Int64)
    to_discard::Dict{Int64, Array{Int64, 1}} = Dict{Int64, Array{Int64}}()
    for z in 1:csp.n_variables
        to_discard[z] = Array{Int64, 1}()
    end
    
    for other_value in csp.domain[new_var]
        if (other_value != instantiation[new_var]) && csp.possible_value[(new_var, other_value)]
            push!(to_discard[new_var], other_value)
            csp.possible_value[(new_var, other_value)] = false
        end
    end
    
    to_test = Set{Int64}()
    for cons in 1:csp.n_constraints
        push!(to_test, cons)
    end
    
    while !isempty(to_test)
        cons = pop!(to_test)
        x = csp.constraint_var1[cons]
        y = csp.constraint_var2[cons]
        for a in csp.domain[x]
            
            if !csp.possible_value[(x, a)]
                continue
            end
            
            supported::Bool = false
            for b in csp.domain[y]
                if !csp.possible_value[(y, b)]
                    continue
                end
                if csp.constraint_satisfaction[cons][a, b]
                    supported = true
                    break
                end
            end
            
            if !supported
                push!(to_discard[x], a)
                csp.possible_value[(x, a)] = false
                for cons_impacted in csp.var2_constraints[x]
                    if csp.constraint_var1[cons] != y
                        push!(to_test, cons_impacted)
                    end
                end
            end
        end
    end
    
    return to_discard
end

AC3 (generic function with 1 method)

In [8]:
function choose_next_variable(csp::CSP, instantiation::Array{Int64, 1})
    best_var::Int64 = 0
    min_domain_size::Int64 = typemax(Int64)
    
    for var in 1:csp.n_variables
        if instantiation[var] != 0
            continue
        end
        
        domain_size::Int64 = 0
        for value in csp.domain[var]
            if csp.possible_value[(var, value)]
                domain_size += 1
            end
        end

        if domain_size < min_domain_size
            best_var = var
            min_domain_size = domain_size
        end
    end
    
    return best_var
end

choose_next_variable (generic function with 1 method)

In [9]:
function backtrack!(
        csp::CSP,
        instantiation::Array{Int64, 1},
        new_var::Int64,
        nodes_explored::Int64,
        look_ahead_method::String
    )
    if !check_feasibility(csp, instantiation, new_var)
        return (false, instantiation, nodes_explored)
    end
    
    new_var = choose_next_variable(csp, instantiation)
    if new_var == 0
        return true, instantiation, nodes_explored
    end
        
    for new_value in csp.domain[new_var]
                
        if !csp.possible_value[(new_var, new_value)]
            continue
        end

        nodes_explored += 1
        instantiation[new_var] = new_value
                    
        if look_ahead_method == "FC"
            discarded = forward_checking(csp, instantiation, new_var)
        elseif look_ahead_method == "MAC3"
            discarded = AC3(csp, instantiation, new_var)
        end
                
        (solution_found, solution, nodes_explored) = backtrack!(
            csp, instantiation, new_var, nodes_explored, look_ahead_method)
        if solution_found
            return (true, solution, nodes_explored)
        end
        
        instantiation[new_var] = 0
        if look_ahead_method != "none"
            undo_look_ahead!(csp, discarded)
        end

    end
    
    return false, instantiation, nodes_explored
end

function backtrack!(csp::CSP, look_ahead_method::String)
    return backtrack!(csp, zeros(Int64, csp.n_variables), 0, 0, look_ahead_method)
end

backtrack! (generic function with 2 methods)

## 2. Applications

### 2.1. N-queens problem

In [10]:
function compatibility_nqueens(var1::Int64, var2::Int64, value1::Int64, value2::Int64)
    # Exclude same row
    if value1 == value2
        return false
    # Exlude same diagonal
    elseif (value1 - value2) == (var1 - var2)
        return false
    # Exclude same antidiagonal
    elseif (value1 - value2) == -(var1 - var2)
        return false
    else
        return true
    end
end

function define_nqueens(n::Int64)
    csp = CSP()
    domain = collect(1:n)
    for i in 1:n
        add_variable!(csp, domain)
    end
    for i in 1:n, j in 1:n
        if (i != j)
            add_constraint!(csp, i, j, compatibility_nqueens)
        end
    end
    return csp
end

function visualize_nqueens(instantiation::Array{Int64})
    n::Int64 = length(instantiation)
    for i in 1:n
        println()
        for j in 1:n
            if instantiation[i] == j
                print(" o")
            else
                print(" .")
            end
        end
    end
end        

visualize_nqueens (generic function with 1 method)

In [11]:
@time nqueens = define_nqueens(20)
@time solution_exists, solution, nodes_explored = backtrack!(nqueens, "MAC3")
print("Nodes explored: ", nodes_explored, " - Solution: ", solution)
visualize_nqueens(solution)

  0.246429 seconds (757.55 k allocations: 38.952 MiB, 5.02% gc time)
  0.294154 seconds (532.63 k allocations: 26.611 MiB, 1.64% gc time)
Nodes explored: 52 - Solution: [1, 3, 5, 14, 17, 4, 16, 7, 12, 18, 15, 19, 6, 10, 20, 11, 8, 2, 13, 9]
 o . . . . . . . . . . . . . . . . . . .
 . . o . . . . . . . . . . . . . . . . .
 . . . . o . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . o . . . . . .
 . . . . . . . . . . . . . . . . o . . .
 . . . o . . . . . . . . . . . . . . . .
 . . . . . . . . . . . . . . . o . . . .
 . . . . . . o . . . . . . . . . . . . .
 . . . . . . . . . . . o . . . . . . . .
 . . . . . . . . . . . . . . . . . o . .
 . . . . . . . . . . . . . . o . . . . .
 . . . . . . . . . . . . . . . . . . o .
 . . . . . o . . . . . . . . . . . . . .
 . . . . . . . . . o . . . . . . . . . .
 . . . . . . . . . . . . . . . . . . . o
 . . . . . . . . . . o . . . . . . . . .
 . . . . . . . o . . . . . . . . . . . .
 . o . . . . . . . . . . . . . . . . . .
 . . . . . . . . . . 

### 2.2 Graph coloring

In [12]:
function read_graph(path::String)
    edges = Array{Tuple{Int64, Int64}, 1}()
    open(path) do file
        for line in eachline(file)
            split_line = split(line)
            if line[1] == 'p'
                n_vertices = split_line[3]
            elseif line[1] == 'e'
                u = parse(Int64, split_line[2])
                v = parse(Int64, split_line[3])
                push!(edges, (u, v))
            end
        end
    end
    return n_vertices, edges
end

read_graph (generic function with 1 method)